<style>
    .title_div {
    padding: 60px;
    text-align: center;
    background: #ffffff;
    }
    #title {
        color: #f2a900;
        font-size: 80px;
    }
    #subtitle {
        color: #4d4d4e;
        font-size: 20px;
    }
</style>

<div class="title_div">
  <h1 id="title">TX</h1>
  <p id="subtitle">Transactions and Script</p>
</div>

<div style="color:red; text-align: center; padding: 60px">
⚠ The following TX examples are just to see how Bitcoin work; don't transmit nothing generated here on the mainnet. ⚠
</div>


## TOC:
* [TxIn](#txin)
* [TxOut](#txout)
* [TX](#tx)
* [Script](#script)
    * [P2PK](#p2pk)
    * [P2PKH](#p2pkh)
    * [P2SH](#p2sh)

In [72]:
import unittest
from io import StringIO, BytesIO

In [73]:
def read_varint(s):
    
    flag = s.read(1)
    if flag == 0xfd:
        # 0xfd next two bytes are the number
        return flag.read(2)
    elif flag == 0xfe:
        # 0xfe next four bytes are the number
        return flag.read(4)
    elif flag == 0xff:
        # 0xff next eight bytes are the number
        return flag.read(8)
    else:
        # the flag is the number
        return flag


def encode_varint(i):
   
    if i < 0xfd:
        return bytes([i])
    elif i < 0x10000:
        return b'\xfd' + i.to_bytes(2, 'litte')
    elif i < 0x100000000:
        return b'\xfe' + i.to_bytes(4, 'litte')
    elif i < 0x10000000000000000:
        return b'\xff' + i.to_bytes(8, 'litte')
    else:
        raise ValueError('integer too large: {}'.format(i))

<a class="anchor" id="txin"></a>
## TxIn

In [74]:
class TxIn:
    def __init__(self, txID, outID, scriptSig = None, sequence = 0xffffffff):
        self.txID = txID # hex string
        self.outIndex = outID # int
        self.scriptSig = scriptSig # 
        self.sequence = sequence
    
    def serialize(self):
        serialized_tx = self.txID[::-1]
        serialized_tx += self.outIndex[::-1]
        if self.scriptSig != None:
            serialized_tx += encode_varint(len(self.scriptSig))
            serialized_tx += self.scriptSig
        serialized_tx += self.sequence[::-1]

        return serialized_tx


    @classmethod
    def parse(cls, input, hex=False):
        if hex:
            stream = input
        else:
            stream = BytesIO(bytes.fromhex(input))
        TxID = stream.read(32)[::-1]
        index = stream.read(4)[::-1]
        script_size = int.from_bytes(read_varint(stream), 'little')
        script = stream.read(script_size)
        secuence = stream.read(4)[::-1]
        return cls(TxID, index, script, secuence)

    def __repr__(self):
        string = '[\n\t'
        string += f'TxID: {self.txID.hex()}\n\t'
        string += f'OutIndex: {self.outIndex.hex()}\n\t'
        if self.scriptSig != None:
            string += f'Script size: {len(self.scriptSig)}\n\t'
            string += f'Script: {self.scriptSig.hex()}\n\t'
        string += f'Sequence: {self.sequence.hex()}\n' + ']'
        return string

<a class="anchor" id="txout"></a>
## TxOut

In [75]:
class TxOut:

    def __init__(self, value, scriptPubKey):
        self.value = value
        self.scriptPubKey = scriptPubKey

    def serialize(self):
        serialized_tx = self.value[::-1]
        serialized_tx += encode_varint(len(self.scriptPubKey))
        serialized_tx += self.scriptPubKey
        
        return serialized_tx

    @classmethod
    def parse(cls, output, hex=False):
        if hex:
            stream = output
        else:
            stream = BytesIO(bytes.fromhex(output))
        value = stream.read(8)[::-1]
        script_size = int.from_bytes(read_varint(stream), 'little')
        script = stream.read(script_size)
        return cls(value, script)

    def __repr__(self):
        string = '[\n\t'
        string += f'Value: {int.from_byes(self.value, "big")}\n\t'
        string += f'Script size: {len(self.scriptPubKey)}\n\t'
        string += f'Script: {self.scriptPubKey.hex()}\n' + ']'
        return string


<a class="anchor" id="tx"></a>
## TX

In [76]:
class TX:
    def __init__(self, inputs, outputs, version=0x00000001 , locktime=0x00000000):
        self.inputs = inputs
        self.outputs = outputs
        self.version = version
        self.locktime = locktime

    def serialize(self):
        serialized_tx = self.version[::-1]
        serialized_tx += encode_varint(len(self.inputs))
        for i in self.inputs:
            serialized_tx += i.serialize()
        
        serialized_tx += encode_varint(len(self.outputs))
        for i in self.outputs:
            serialized_tx += i.serialize()

        serialized_tx += self.locktime[::-1]
        return serialized_tx

    @classmethod
    def parse(cls, tx):
        stream = BytesIO(bytes.fromhex(tx))

        version = stream.read(4)[::-1]

        input_count = int.from_bytes(read_varint(stream), 'little')
        inputs = []
        for i in range(input_count):
            inputs.append(TxIn.parse(stream, True))

        output_count = int.from_bytes(read_varint(stream), 'little')
        outputs = []
        for i in range(output_count):
            outputs.append(TxOut.parse(stream, True))
        locktime = stream.read(4)[::-1]

        return cls(inputs, outputs, version, locktime)




TX.parse('01000000017967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff014baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac00000000')

In [77]:
class TestTX(unittest.TestCase):
    def test_TxIn(self):
        input='7967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff'
        aux = TxIn.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

    def test_TxOut(self):
        input='4baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac'
        aux = TxOut.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

    def test_TX(self):
        input='01000000017967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff014baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac00000000'
        aux = TX.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_TX (__main__.TestTX) ... ok
test_TxIn (__main__.TestTX) ... ok
test_TxOut (__main__.TestTX) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
